In [ ]:
!pip install nest-asyncio

In [ ]:
import os
env_data = {
    "OPENAI_API_KEY":"<your-api-key>",
    "DOCUMENTDB_CONNECTION_STRING":r"<your-connection-string>"
    }

os.environ.update(env_data)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Assume that you are passionate about baking cakes, and you are looking for some recipes for your next cake. Fortunately you have a large database of cakes. Which cake would be best suited for you?

In [ ]:
from langchain_core.documents import Document
recipes = [
    Document(page_content="In order to make the famous langchain chocolate cake Blend dark chocolate and cocoa powder for a rich chocolate cake base.", metadata={"recipeId": "101"}),
    Document(page_content="In order to make the famous langchain cheese cake Combine cream cheese, sugar, and eggs to prepare the cheesecake batter.", metadata={"recipeId": "102"}),
    Document(page_content="In order to make the famous langchain orange cake Stir together flour, fresh orange zest, and orange juice for a zesty orange cake.", metadata={"recipeId": "103"}),
    Document(page_content="In order to make the famous langchain banan cake Mix mashed bananas with cinnamon and nutmeg for a spiced banana cake.", metadata={"recipeId": "104"}),
    Document(page_content="In order to make the famous langchain cardmom cake Whisk eggs, almond flour, and ground cardamom for a fragrant cardamom cake.", metadata={"recipeId": "105"})
]

In [ ]:
# You can create the collection using afrom_documents

In [ ]:
from langchain_community.vectorstores.async_documentdb import AsyncDocumentDBVectorSearch
from langchain_community.embeddings.openai import OpenAIEmbeddings
from motor.motor_asyncio import AsyncIOMotorClient
import os

connection_string = os.environ.get("DOCUMENTDB_CONNECTION_STRING")

namespace = "cookbook.cakes"
db_name, collection_name = namespace.split(".")
motor_client = AsyncIOMotorClient(connection_string)
collection = motor_client[db_name][collection_name]
if collection_name in await motor_client[db_name].list_collection_names():
    await collection.drop()
index_name = "cakes"

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Vector dimension for the index similarity
dimensions = 1536
vectorstore = await AsyncDocumentDBVectorSearch.afrom_documents(
    recipes,
    embedding_model,
    collection=collection,
    index_name=index_name
)

In [ ]:
# Lets create the index ove the documents
from langchain_community.vectorstores.async_documentdb import DocumentDBSimilarityType


await vectorstore.acreate_index(dimensions, DocumentDBSimilarityType.COS)

In [ ]:
#Lets create a chain that can call the vectorstore
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough

# We define a prompt here, `recipe` is the placeholder for the most suitable recipe
prompt = """
You are an expert conditor, Please recommend a suitable cake recipe from langchain.
{recipe}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [('system', prompt),
    ("human","{message}")]
)

# Vectorstores can also create a retriever object which could be used inside a chain to retrieve relevant objects.
# Additionaly, you can add aggregation commands such as $match to this call to filter on specific fields.
# for example
# retriever = vectorstore.as_retriever([{
                            # "$match": {
                            # "recipeId": 123
                            # }
                            # }])

retriever = vectorstore.as_retriever()

# We would need a small helper function to turn the results of the search to a string
def format_docs(docs) -> str:
        """Helper function to format docs as input for RAG chaing
        Args:
            docs (list): list of Document objects retrieved from vector store
        """
        return "\n\n".join(doc.page_content for doc in docs)
    
# We need a function to extract the message as string from the input object, I recommend you to check how the input looks like by printing it. You can also write this expression as a lambda function
def input_formatter(_input) -> str:
    return _input["message"]

# Let's create the chain
llm = ChatOpenAI()
chain = (
    RunnablePassthrough.assign(
        recipe=input_formatter | retriever | format_docs
        )
    | prompt_template
    | llm
)


In [ ]:
# Now Let's ask our chain to find us a recipe with cheescake
message = dict(
    message="Recommend me a good cheescake"
)
res = await chain.ainvoke(input=message)
print(res.content)


>>> I recommend trying the famous Langchain cheesecake recipe. To make it, combine cream cheese, sugar, and eggs to prepare the cheesecake batter. You can also add a hint of vanilla extract for extra flavor. Bake the cheesecake until it is set and then chill it in the fridge before serving. Enjoy this delicious and creamy dessert!

In [ ]:
message = dict(
    message="Recommend me a recipe with banana"
)
res = await chain.ainvoke(input=message)
print(res.content)

>>> I recommend trying the famous langchain banana cake recipe! To make this delicious spiced banana cake, mix mashed bananas with cinnamon and nutmeg for a flavorful twist. It's a perfect treat for any occasion and is sure to be a crowd-pleaser. Enjoy baking!